# imports:

In [ ]:
from os import listdir,mkdir
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist
from Feature_phenotype_functions import common_processing_feature_phenotype_matrices, \
          editSampleNames,process_sample_matrix,compute_distance_matrix_general


In [ ]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

# load files:


In [ ]:
#load BD-reg num file
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD_date_correctReg_userID'
BD_date_correctReg_userID=pd.read_pickle(file1)

In [ ]:
#load relationship file from Noa:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/participant genetic relations.xlsx'
participantGeneticRelations=pd.read_excel(file1)

In [ ]:
participantGeneticRelations.head()

In [ ]:
BD_date_correctReg_userID.head()

# add correct BD numbers

In [ ]:
BD_REG= BD_date_correctReg_userID[['DnaID','correct registration code']]

In [ ]:
GenRelationsWithBDs=pd.merge(participantGeneticRelations,BD_REG,how='left',left_on='Female',right_on='correct registration code')
GenRelationsWithBDs.head()

In [ ]:
GenRelationsWithBDs=GenRelationsWithBDs.rename(columns={'DnaID':'Female_BD'})

In [ ]:
GenRelationsWithBDs=pd.merge(GenRelationsWithBDs,BD_REG,how='left',left_on='Male',right_on='correct registration code')
GenRelationsWithBDs.head()

In [ ]:
GenRelationsWithBDs=GenRelationsWithBDs.rename(columns={'DnaID':'Male_BD'})

In [ ]:
file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/GenRelationsWithBDs.xlsx'
GenRelationsWithBDs.to_excel(file2)

# after manuall editing with excel, add more BD numbers to unidentified regNums:

In [ ]:
#load GenRelation file:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/GenRelationsWithBDs_final.xlsx'
GenRelationsWithBDs_final=pd.read_excel(file1)
GenRelationsWithBDs_final.head()

In [ ]:
#load BD-FD converter file:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/updatedBDandFDlists/final_BD_FD_converter_mergedOnYear_11032018'
final_BD_FD_converter_mergedOnYear_11032018=pd.read_pickle(file1)
final_BD_FD_converter_mergedOnYear_11032018.head()

In [ ]:
onlyBDandRegNums=final_BD_FD_converter_mergedOnYear_11032018[['BD','correct registration code']]
print len(onlyBDandRegNums)
onlyBDandRegNums2=onlyBDandRegNums.drop_duplicates(onlyBDandRegNums)
print len(onlyBDandRegNums2)

In [ ]:
GenRelationsWithBDs_moreBDs=pd.merge(GenRelationsWithBDs_final,onlyBDandRegNums2,how='left',
                                     left_on='correctRegNum',right_on='correct registration code')
GenRelationsWithBDs_moreBDs.head()

In [ ]:
GenRelationsWithBDs_moreBDs.tail()

In [ ]:
GenRelationsWithBDs_moreBDs[GenRelationsWithBDs_moreBDs['DnaID']=='BD551']

In [ ]:
print len(GenRelationsWithBDs_moreBDs[GenRelationsWithBDs_moreBDs['DnaID'].isnull()])
GenRelationsWithBDs_moreBDs[GenRelationsWithBDs_moreBDs['DnaID'].isnull()]

In [ ]:
GenRelationsWithBDs_moreBDs[GenRelationsWithBDs_moreBDs['DnaID']=='BD551']

In [ ]:
GenRelationsWithBDs_moreBDs2=GenRelationsWithBDs_moreBDs
GenRelationsWithBDs_moreBDs2['DnaID']=np.where(GenRelationsWithBDs_moreBDs2['DnaID'].isnull(),GenRelationsWithBDs_moreBDs2['BD'],
                                                      GenRelationsWithBDs_moreBDs2['DnaID'])

In [ ]:
print len(GenRelationsWithBDs_moreBDs[GenRelationsWithBDs_moreBDs['DnaID'].isnull()])
print len(GenRelationsWithBDs_moreBDs2[GenRelationsWithBDs_moreBDs2['DnaID'].isnull()])

In [ ]:
GenRelationsWithBDs_moreBDs[GenRelationsWithBDs_moreBDs['DnaID']=='BD551']

In [ ]:
GenRelationsWithBDs_moreBDs2[GenRelationsWithBDs_moreBDs2['DnaID']=='BD551']

In [ ]:
GenRelationsWithBDs_moreBDs2=GenRelationsWithBDs_moreBDs2[['correctRegNum','DnaID','Relations','Gender']]

In [ ]:
#Save updated GenRelation file:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/GenRelationsWithBDs_moreBDs2.xlsx'
GenRelationsWithBDs_moreBDs2.to_excel(file1)